INGESTA PARAGUAY

QUERY ORIGEN

In [ ]:
WITH InitialSales AS (
    SELECT  
    a.cIDCompany Company_DwID
    ,a.cIDInvoice Invoice_DwID
    ,a.cSerie Serie
    ,a.cForm Form
    ,a.cIDCustomer Customer_DwID 
    ,a.cIDBranchInvoice BranchDwID
    ,a.cIDTerritory Territory_DwID
    ,a.dEmission Emission
    ,a.xChannelDistribution ChannelDistribution 
    ,a.xReturnReason InvoiceCancelReason
    --,a.xInvoiceCancelReason InvoiceCancelReason
    ,b.cidOrder Order_DwID
    ,a.xInvoiceType InvoiceType 
    ,e.cIDProduct Product_DwID
    ,e.nAmount Amount
    ,e.nUnitValue UnitValue
    ,e.nTotalValue TotalValue
    ,e.MC1LastUpdate
    ,e.nFreegoods Freegoods
    ,g.nSeq Seq
    ,g.nValue Value
    ,g.npercentage
    ,g.cIDPricing Pricing_DwID
    ,g.cIDStep Step_DwID
    ,f.xIDReasonReturnStock ReasonReturnStock_DwID
    ,h.lCalculated Calculated
    ,h.cIDPricingFormationRule PricingFormationRule_DwID
    from CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_Invoice a
    --################# order invoice #########################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_OrderInvoice 
    ) b 
    on a.cIDCompany = b.cIDCompany
    and a.cIDInvoice = b.cIDInvoice
    and a.cForm = b.cform
    and a.cSerie = b.cSerie
    and a.cIDBranchInvoice =b.cIDBranchInvoice
    AND a.mc1Enabled = 1
    and b.mc1enabled = 1
    --################## invoice item #####################
    inner join (select *
    FROM CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_InvoiceItem
    ) e 
    on e.cIDCompany = a.cIDCompany 
    and e.cIDInvoice = a.cIDInvoice 
    and e.cForm = a.cform
    and e.cSerie = a.cSerie
    AND e.cIDBranchInvoice=a.cIDBranchInvoice 
    and e.mc1enabled = 1
    --################ invoice item detail #######################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_InvoiceItemDetail 
    ) g
    on g.cIDCompany = e.cIDCompany
    and g.cIDInvoice = e.cIDInvoice
    and g.cForm = e.cform
    and g.cSerie = e.cSerie
    AND g.cIDBranchInvoice = e.cIDBranchInvoice
    and g.nSeq = e.nSeq --igualando pasos 
    and g.cidProduct = e.cidProduct --igualando los productos de factura y order
    and g.mc1enabled = 1
    and g.cIDStep in ('00100','00130','00160','00200','00250','00450','00500','00700','00900','00900','00600') --steps para llegar a la venta neta 
    --################### MC1_MC_OrderExt ########################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_OrderExt
    ) f 
    on f.cIDOrder = b.cIDOrder
    and f.cIDCompany = b.cIDCompany
    and f.mc1enabled = 1
    --#################### REQUEST EMITES ######################
    INNER JOIN (SELECT * 
    FROM CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_InvoiceRequestEmites
    ) m
    on a.cIDCompany = m.cIDCompany
    and a.cIDInvoice = m.cIDInvoice
    and a.cForm = m.cform
    and a.cSerie = m.cSerie
    and a.cIDBranchInvoice =m.cIDBranchInvoice
    and m.mc1enabled = 1
    and UPPER(m.xInvoiceRequestStatus)= 'SUCCEEDED' 
    --################### PRICING_RULE ###################
    inner join (
    select *
    from CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_PricingFormationRule
    ) h
    on g.cIDStep = h.cIDStep
    and g.cIDPricing = h.cIDPricing
    and g.cIDCompany = h.cIDCompany
    and g.mc1enabled = 1
    and h.mc1enabled = 1
    and upper(h.cIDPricingFormationRule) != 'UNITVALUECALCULATED' --se repite el paso 00100 y parece aumentar el valor 
    where a.xInvoiceType not in ('WTM002','WTM005','ZOGRD','ZOGR') --Ordertype no buscados a fin de ver os nuevos
    and format(a.dEmission , 'yyyy-MM-dd') between '{inicio}' and '{fin}' --informacion de un dia  
    and e.nUnitValue >= 0
),
ValidOrders AS (
    SELECT distinct b.cIDOrder, b.cIDProduct,a.cIDBranchinvoice 
    from CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_OrderInvoice a
    inner join CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_OrderItem b 
    on a.cIDOrder = b.cIDOrder 
    and a.cIDCompany = b.cIDCompany 
    and a.cIDBranchInvoice = b.cIDBranchinvoice
    where 1=1
    and a.mc1Enabled = 1
    and b.mc1Enabled = 1
    and format(a.mc1lastupdate , 'yyyy-MM-dd') between '{inicio_orden}' and '{fin}' --informacion de un dia 
),
ValidSales AS (
    SELECT 
        a.* 
    FROM InitialSales a
    INNER JOIN ValidOrders b ON
        a.Order_DwID = b.cIDOrder AND
        a.Product_DwID = b.cIDProduct AND
        a.BranchDwID = b.cIDBranchInvoice
    WHERE b.cIDOrder is not null
),
FirstDeduplicates AS(
    SELECT
        *
    FROM (
        SELECT
        a.*,
        ROW_NUMBER() OVER (PARTITION_BY a.BranchDwID, a.ChannelDistribution, 
            a.Company_DwID, a.Customer_DwID, a.Emission, a.Form, a.FreeGoods, a.Invoice_DwID, a.InvoiceCancelReason, a.InvoiceType
            a.Pricing_DwID, a.Product_DwID, a.Seq, a.Serie, a.Step_DwID, a.Territory_DwID, ORDER BY a.mc1LastUpdate DESC, a.ReasonReturnStock_DwID DESC) as rn
    FROM ValidSales a
    ) sub
    WHERE rn = 1
),
Transformations AS (
    SELECT
        a.*,
        CASE WHEN a.Step_DwID = '00100' THEN [Value] ELSE 0 END AS Valor_Bruto,
        CASE WHEN a.Step_DwID = '00400' THEN [Value] ELSE 0 END AS Promocion_ML,
        CASE WHEN a.Step_DwID = '00600' THEN [Value] ELSE 0 END AS Descuento_ML,
        CASE WHEN a.Step_DwID = '00150' THEN [Value] ELSE 0 END AS Escalafc_ML,
        CASE 
            WHEN a.Step_DwID = '00200' AND a.Freegoods = 0 THEN a.[Value] 
            WHEN a.Step_DwID = '00100' AND a.Freegoods = 1 THEN a.[Value] 
            ELSE 0
        END AS Facdsl_ML,
        CASE WHEN a.Step_DwID = '01300' THEN [Value] ELSE 0 END AS Impuesto1,
        CASE WHEN a.Step_DwID = '01100' THEN [Value] ELSE 0 END AS Impuesto2,
        CASE WHEN a.Step_DwID = '01400' THEN [Value] ELSE 0 END AS Venta_Impuestos
    FROM FirstDeduplicates a
),
GroupedSales AS (
    SELECT
        Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        Territory_DwID,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Impuesto1) AS Impuesto1,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(Impuesto2) AS Impuesto2,
        SUM(Escalafc_ML) AS Escalafc_ML
    FROM Transformations
    GROUP BY
        Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        Territory_DwID,
        UnitValue
),
GroupesSales2 AS (
      SELECT
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Serie,
        Territory_DwID,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Amount) AS Amount,
        SUM(Impuesto1) AS Impuesto1,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(Impuesto2) AS Impuesto2,
        SUM(Escalafc_ML) AS Escalafc_ML
    FROM Transformations
    GROUP BY
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Serie,
        Territory_DwID,
        UnitValue,
)
TransformationFACDSL AS(
    SELECT
        a.*,
        CASE WHEN a.Facdsl_ML <> 0 THEN a.Valor_Bruto * a.Facdsl_ML ELSE 0 END as Facdsl_ML
    FROM GroupesSales2 a
)
InvoiceItemDetailJoin AS (
    SELECT a.cIDCompany,a.cIDBranchInvoice ,a.cSerie ,a.cIDInvoice,a.cForm ,a.cIDProduct
    ,format(a.MC1LastUpdate,'yyyy-MM-dd hh:mm') MC1LastUpdate ,c.nfreegoods
    ,a.nValue Value2
    --,sum(a.Value) Value2 ,COUNT(*) no_registros
    FROM CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_InvoiceItemDetail a
    inner join (SELECT * FROM CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_Invoice 
    WHERE MC1Enabled = 1
    and format(DEmission , 'yyyy-MM-dd') between '{inicio}' and '{fin}' --informacion de un dia  
    and xInvoiceType not in ('WTM002','WTM005','ZOGRD','ZOGR') ) b
    on a.cIDBranchInvoice = b.cIDBranchInvoice
    and a.cSerie = b.cSerie
    and a.cIDInvoice = b.cIDInvoice
    and a.cForm = b.cForm
    inner join (SELECT * FROM CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_Invoiceitem
    WHERE MC1Enabled = 1 ) c
    on a.cIDBranchInvoice = c.cIDBranchInvoice
    and a.cSerie = c.cSerie
    and a.cIDInvoice = c.cIDInvoice
    and a.cForm = c.cForm
    and a.cIDProduct = c.cIDProduct
    and a.nSeq = c.nSeq
    where a.cIDStep = '00130'
    and a.MC1Enabled = 1
    group by a.cIDCompany,a.cIDBranchInvoice ,a.cSerie ,a.cIDInvoice,a.cForm ,a.cIDProduct
    ,format(a.MC1LastUpdate,'yyyy-MM-dd hh:mm'),c.nfreegoods
    ,a.nValue
),
InvoiceItemDetailDeduplicated AS (
    SELECT distinct
        a.*
    FROM InvoiceItemDetailJoin a
)
InvoiceDetailGenerated AS (
    SELECT 
        Company_DwID,
        BranchInvoice_DwID,
        Serie,
        Invoice_DwID,
        Form,
        Product_DwID,
        MC1LastUpdate,
        FreeGoods,
        CASE WHEN FreeGoods = 1 THEN 0 ELSE nValue END AS nValue
    FROM InvoiceItemDetailDeduplicated
),
InvoiceDetailGrouped AS (
    SELECT
        Company_DwID,
        BranchInvoice_DwID,
        Form,
        Invoice_DwID,
        Product_DwID,
        Serie,
        SUM(nValue) AS facdet
    FROM InvoiceDetailGenerated
    GROUP BY 
        Company_DwID,
        BranchInvoice_DwID,
        Form,
        Invoice_DwID,
        Product_DwID,
        Serie
),
FinalJoin AS (
    SELECT 
        a.*,
        b.facdet
    FROM GroupedSales a
    INNER JOIN InvoiceDetailGrouped b ON
        a.Company_DwID = b.Company_DwID AND
        a.BranchDwID = b.BranchInvoice_DwID AND
        a.Product_DwID = b.Product_DwID AND
        a.Serie = b.Serie AND
        a.Form = b.Form AND
        a.Invoice_DwID = b.Invoice_DwID
),
AddColumns AS (
    SELECT 
        a.Amount_Sum,
        a.BranchDwID,
        a.ChannelDistribution,
        a.Company_DwID,
        a.Customer_DwID,
        a.Descuento_ML,
        a.Escalafc_ML,
        a.Facdsl_ML,
        a.Freegoods,
        a.InvoiceCancelReason,
        a.Invoice_DwID,
        a.InvoiceType,
        a.Order_DwID,
        a.Promocion_ML,
        a.ReasonReturnStock_DwID,
        a.Seq,
        a.Serie,
        a.Territory_DwID,
        a.UnitValue,
        a.Valor_Bruto,
        a.Venta_Impuestos,
        a.Impuesto1,
        a.Impuesto2,
        CONCAT(a.Serie, '-', a.Invoice_DwID) AS Numero,
        a.ChannelDistribution AS Cod_Linea_ruta,
        a.Company_DwID AS Pais,
        a.InvoiceType AS AG19ID,
        a.Customer_DwID AS Codigo_Cliente,
        a.BranchDwID AS Codigo_Agencia,
        a.Territory_DwID AS Codigo_Ruta,
        a.Product_DwID AS Codigo_Producto,
        'FALSE' AS FACCON,
        CASE
            WHEN a.InvoiceType = 'ZBON' THEN 0 - facdet
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' THEN 0 - facdet
            ELSE facdet
        END AS facdet,
        CASE 
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' THEN a.ReasonReturnStock_DwID
            WHEN NOT (a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX') THEN a.InvoiceCancelReason
            ELSE 'REVISAR'
        END AS Id_Motivo,
        FORMAT(CAST(a.Emission AS date), 'yyyy-MM-dd') AS Fecha,
        REPLACE(a.Territory_DwID, a.BranchDwID, '') AS Territory_DwID_Short
    FROM FinalJoin a
),
JoinWithSubchannel AS (
    SELECT *
    FROM (
        SELECT 
            a.*,
            b.cidbranchinvoice,
            ROW_NUMBER() OVER (PARTITION BY b.cidbranchinvoice, b.cidcustomer ORDER BY a.mc1Enabled DESC, a.mc1LastUpdate DESC) AS rn
        FROM CBRTM3XPOLCD22_LASC1.CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_CustomerDivisionSectorExt a
        LEFT JOIN CBRTM3XPOLCD22_LASC1.CBRTM3XPOLCD22_LASC1.GRO_BPR_RTM.VW_MC1_CustomerDivisionSector b ON
            a.cidcustomer = b.cidcustomer
        WHERE 
            b.cidbranchinvoice IS NOT NULL
    ) sub
    WHERE rn = 1
),
FinalTablaSubchannel as 

SELECT * FROM JoinWithSubchannel;
